## NLP ASSIGNMENT #2-J033

In [65]:
import gensim.downloader as api
import pandas as pd
import re
from gensim.models import Word2Vec
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np


## PART 1


In [3]:
model=api.load("word2vec-google-news-300")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [22]:
words=['horse','forest','stock','market','gold']

for word in words:
    similar_words=model.most_similar(word,topn=5)
    print(f'Words similar to :{word}')
    for similar_word, similarity in similar_words:
        print(f'{similar_word}:{similarity:.4f}', end="|")
    print()
        

Words similar to :horse
horses:0.8654|racehorse:0.7524|stallion:0.7200|thoroughbred:0.7159|horseman:0.6846|
Words similar to :forest
forests:0.8682|forested:0.7113|forestland:0.6870|forestry:0.6760|Forests:0.6707|
Words similar to :stock
shares:0.7760|stocks:0.6025|Shares:0.5808|Stock:0.5691|stockmarket:0.5496|
Words similar to :market
markets:0.7666|marketplace:0.6955|themarket:0.5991|maket:0.5921|mkt:0.5859|
Words similar to :gold
silver:0.8313|precious_metal:0.7142|domed_Moussa:0.6745|Nastia_deserved:0.6672|Gold:0.6448|


In [51]:
analogies = [
    ('king', 'man', 'woman', 'queen'),
    ('paris', 'france', 'germany', 'berlin'),
    ('Football', 'Ronaldo', 'Virat', 'messi')
]

for a,b,c, expected in analogies:
    result=model.most_similar(positive=[c,a],negative=[b],topn=1)
    predicted=result[0][0]
    print(f'{a}-{b}+{c}={predicted} (expected:{expected})')

king-man+woman=queen (expected:queen)
paris-france+germany=berlin (expected:berlin)
Football-Ronaldo+Virat=Cricket (expected:messi)


## PART 2

In [79]:
df=pd.read_csv("C://Users//hones//Downloads//archive//IMDB Dataset.csv")

In [80]:
#Perform text EDA
print("Dataset Shape:", df.shape)
print("Dataset Columns:", df.columns)
print("Class Distribution:\n", df['sentiment'].value_counts())
print("First 5 Reviews:\n", df['review'].head())

Dataset Shape: (50000, 2)
Dataset Columns: Index(['review', 'sentiment'], dtype='object')
Class Distribution:
 sentiment
positive    25000
negative    25000
Name: count, dtype: int64
First 5 Reviews:
 0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object


In [81]:
#Clean data
def clean_text(text):
    text = text.lower()  # Lowercase the text
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.split()  # Tokenize text
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    text = ' '.join(text)
    return text


In [82]:
df['clean_review'] = df['review'].apply(clean_text)


In [83]:
print("First 5 Cleaned Reviews:\n", df['clean_review'].head())

First 5 Cleaned Reviews:
 0    one reviewers mentioned watching oz episode yo...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
3    basically theres family little boy jake thinks...
4    petter matteis love time money visually stunni...
Name: clean_review, dtype: object


In [84]:
#Tokenization
df['tokens'] = df['clean_review'].apply(lambda x: x.split())

In [94]:
model1 = Word2Vec(
    sentences=df['tokens'], 
    vector_size=100, window=5, 
    min_count=5, 
    sg=1)
#Change sg (1 for skipgram, 0 for cbow)

In [95]:
#Vectorization
def vectorize_review(review, model):
    vector = [model.wv[word] for word in review if word in model.wv]
    return np.mean(vector, axis=0) if len(vector) > 0 else np.zeros(model.vector_size)


In [96]:
X = df['tokens'].apply(lambda x: vectorize_review(x, model1))
X = list(X)
X = np.array(X)
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [98]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [99]:
y_pred = clf.predict(X_test)

In [100]:
print(f'Accuracy on Skipgram: {accuracy_score(y_test, y_pred)}')

Accuracy on Skipgram: 0.8366


In [103]:
#Pretrained word2vec model
X = df['tokens'].apply(lambda x: vectorize_review(x,model))
X = list(X)
X = np.array(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

#Evaluation
y_pred = clf.predict(X_test)
print("Accuracy with Pretrained Word2Vec Vectors:", accuracy_score(y_test, y_pred))

Accuracy with Pretrained Word2Vec Vectors: 0.8454


In [107]:
#Metrics
metrics = {
    'Model': ['Custom Skip-gram', 'Custom CBoW', 'Pretrained Word2Vec'],
    'Accuracy': ["85.0%", "83.7%", "84.54%"]  # Example accuracies, replace with actual values
}

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

                 Model Accuracy
0     Custom Skip-gram    85.0%
1          Custom CBoW    83.7%
2  Pretrained Word2Vec   84.54%
